### ROBOT TELEOP

- WASD for the xy plane, 
- RF for the z-axis, 
- QE for tilt, and 
- ARROWs for the rest of rthe otations. 

- SPACEBAR will change your gripper's statE 
- Z key will reset your environment 

#### 1. Library

In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.y_env import SimpleEnv
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

/home/oguz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Setup Sim Environment

In [2]:
SEED = 0 
# SEED = None <- to randomize the object positions
REPO_NAME = 'omy_pnp'
NUM_DEMO = 1 # Number of demonstrations to collect
xml_path = './asset/example_scene_y.xml'
PnPEnv = SimpleEnv(xml_path, seed = SEED, state_type = 'joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[24] n_qvel:[22] n_qacc:[22] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[21]
 [0/21] [world] mass:[0.00]kg
 [1/21] [front_object_table] mass:[1.00]kg
 [2/21] [camera] mass:[0.00]kg
 [3/21] [camera2] mass:[0.00]kg
 [4/21] [camera3] mass:[0.00]kg
 [5/21] [link1] mass:[2.06]kg
 [6/21] [link2] mass:[3.68]kg
 [7/21] [link3] mass:[2.39]kg
 [8/21] [link4] mass:[1.40]kg
 [9/21] [link5] mass:[1.40]kg
 [10/21] [link6] mass:[0.65]kg
 [11/21] [camera_center] mass:[0.00]kg
 [12/21] [tcp_link] mass:[0.32]kg
 [13/21] [rh_p12_rn_r1] mass:[0.07]kg
 [14/21] [rh_p12_rn_r2] mass:[0.02]kg
 [15/21] [rh_p12_rn_l1] mass:[0.07]kg
 [16/21] [rh_p12_rn_l2] mass:[0.02]kg
 [17/21] [body_obj_mug_5] mass:[0.00]kg
 [18/21] [object_mug_5] mass:[0.08]kg
 [19/21] [body_obj_plate_11] mass:[0.00]kg
 [20/21] [object_plate_11] mass:[0.10]kg
body_total_mass:[13.27]kg

n_geom:[83]
geom_names:['floor', 

#### 3. Teleop Robot

In [3]:
action = np.zeros(7)
episode_id = 0
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            PnPEnv.reset(seed = SEED)
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if reset:
            PnPEnv.reset(seed=SEED)

        # Get the end-effector pose and images
        ee_pose = PnPEnv.get_ee_pose()
        agent_image,wrist_image = PnPEnv.grab_image()

        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)

        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))

        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)

        joint_q = PnPEnv.step(action)

        PnPEnv.render(teleop=True)

PnPEnv.env.close_viewer()